In [ ]:
%load_ext autoreload
%autoreload 2
from logging import getLogger
from lewidi_lib import enable_logging
import pandas as pd
import seaborn as sns
pd.set_option("display.max_colwidth", 100)
enable_logging()
logger = getLogger(__name__)
sns.set_context("talk")

In [ ]:
from lewidi_lib import assert_path_exists, preds_file

file = preds_file(
    dataset="CSC",
    split="train",
    template="60",
    model_id="Qwen/Qwen3-32B",
    run_name="1000ex_10loops",
)
rdf = pd.read_parquet(assert_path_exists(file))

In [ ]:
from lewidi_lib import assign_cols_perf_metrics_softlabel, join_dataset, process_rdf
rdf = process_rdf(rdf, response_contains_steps=True)
joint_df = join_dataset(rdf)
joint_df = assign_cols_perf_metrics_softlabel(joint_df)

In [ ]:
from lewidi_lib import assign_col_diversity, avg_pairwise_ws_loss

n_preds_by_problem = joint_df.groupby("dataset_idx", as_index=False).size()
diversity_data = n_preds_by_problem.query("size > 1").merge(joint_df, on="dataset_idx")
answer_diversity = diversity_data.groupby("dataset_idx", as_index=False).agg(
    avg_pairwise_ws_loss=("pred", avg_pairwise_ws_loss),
    avg_ws_loss=("ws_loss", "mean"),
)
answer_diversity = assign_col_diversity(answer_diversity)

q5_diversity_ids = answer_diversity.query("diversity == 'Q5'")["dataset_idx"].unique()
joint_df_q5 = joint_df.query("dataset_idx in @q5_diversity_ids")

In [ ]:
answer_diversity.groupby("diversity", as_index=False, observed=True).agg(
    avg_ws_loss=("avg_ws_loss", "mean"),
    avg_pairwise_ws_loss=("avg_pairwise_ws_loss", "mean"),
    n_examples=("dataset_idx", "count"),
).round(2)

In [ ]:
# answer_diversity.query("diversity == 'Q5'")[["dataset_idx", "diversity"]].to_parquet("high_diversity_ids.parquet")

# """
# copy (
#     select rdf.*
#     from (select * from '../../1000ex_10loops/preds/responses.parquet') as rdf
#     join 'high_diversity_ids.parquet' as ids on rdf.dataset_idx = ids.dataset_idx
#     )
# to 'responses.parquet';
# """

# Load Ratings

In [ ]:
from lewidi_lib import (
    assign_col_response_parsed,
    discard_na_response_rows,
    process_ratings,
)
import numpy as np
from prm800k import mapping


ratings_file = "/Users/tomasruiz/datasets/dss_home/lewidi-data/sbatch/di38bec/Qwen_Qwen3-32B/set2/t60/CSC/train/1000ex_10loops_high_diversity/judge/gemini-2.5-flash/responses.parquet"
ratings = pd.read_parquet(ratings_file)
logger.info("Loaded %d ratings for %d differnt dataset_idxs", len(ratings), ratings["dataset_idx"].nunique())
ratings = discard_na_response_rows(ratings)
ratings = assign_col_response_parsed(ratings)
ratings = process_ratings(
    ratings, operation=np.mean, cat_mapping=mapping(ok=0.0, bad=0)
)
ratings = ratings[
    ["dataset_idx", "run_idx", "step_ratings", "score", "reasoning", "response_parsed"]
]
ratings.rename(columns={"reasoning": "judge_reasoning"}, inplace=True)

In [ ]:
joint_df_q5 = joint_df_q5.merge(ratings, on=["dataset_idx", "run_idx"], how="left")
joint_df_q5 = discard_na_response_rows(joint_df_q5, col="score")

# Problem-Level Correlation

In [ ]:
from lewidi_lib import bootstrap_avg
def corr(df):
    coeff = np.corrcoef(df["score"], df["ws_loss"])[0, 1]
    return coeff
corrs = joint_df_q5.groupby("dataset_idx")[["score", "ws_loss"]].apply(corr).fillna(0)
bootstrap_avg(corrs)

# BoN Loss Stats

In [ ]:
len(joint_df_q5.drop_duplicates(subset=["dataset_idx", "run_idx"]))

In [ ]:
joint_df_q5[["ws_loss", "score"]].apply(bootstrap_avg)

In [ ]:
joint_df_q5.groupby("dataset_idx")[["ws_loss", "score"]].mean().apply(bootstrap_avg)

In [ ]:
bon = joint_df_q5.loc[joint_df_q5.groupby("dataset_idx")["score"].idxmax()]
bon[["ws_loss", "score"]].apply(bootstrap_avg)

In [ ]:
oracle = joint_df_q5.loc[joint_df_q5.groupby("dataset_idx")["ws_loss"].idxmin()]
oracle[["ws_loss", "score"]].apply(bootstrap_avg)

# Individual Examples

In [ ]:
max_loss = joint_df_q5.loc[joint_df_q5.groupby("dataset_idx")["ws_loss"].idxmax()]
row = max_loss.iloc[0]
print(row["response"])